<p style="font-weight:bold;"> <span style="font-size: 36px"> Test Reports </span> </p>

In [1]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

# BenchMarks

In [1]:
var pathToBm = "./Data/ReportBenchmarks/";

In [1]:
var argsArray = new ImportArgs[]{
    new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, null),
    new ImportArgs ("CH", 2021, 3, Periodicity.Quarterly, null, null)
    };

In [1]:
public async Task ExportBenchmarks(ImportArgs args)
{
    await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new {ReportingNode = args.ReportingNode, Year= args.Year, Month = args.Month, Scenario = args.Scenario});
    var computedVariables = (await DataSource.Query<IfrsVariable>().ToArrayAsync()).ToDictionaryGrouped(x => x.EstimateType, x => x.ToArray());

    foreach (var estimateType in computedVariables.Keys)
    {
        await Export.ToCsv(pathToBm+"BM_"+args.ReportingNode+"_"+args.Year.ToString()+"_"+args.Month.ToString()+"_"+estimateType)
                                    .WithTable<ImportArgs>(tableConfig => tableConfig
                                                                    .AtBeginning()
                                                                    .WithName(Main)
                                                                    .WithSource(source => args.RepeatOnce().AsQueryable())
                                                                    .WithColumn(x => x.Periodicity, x => x.Delete())
                                                                    .WithColumn(x => x.ImportFormat, x => x.Delete())
                                                    )
                                    .WithTable<IfrsVariable>(tableConfig => tableConfig
                                                                  .WithSource(source => computedVariables[estimateType].AsQueryable())
                                                                  .WithName(estimateType)
                                                                  .WithColumn(x => x.Partition, x => x.Delete())
                                                                  .WithColumn(x => x.Id, x => x.Delete())
                                                                 )
                                    .ExecuteAsync();
    }
}

In [1]:
foreach (var args in argsArray)
        await ExportBenchmarks(args);